In [1]:
# The required libraries are imported here
import os
from pathlib import Path
import numpy as np

from PIL import Image, ImageFilter
from matplotlib import pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import cv2
import skimage.exposure as exposure
from scipy import ndimage
% matplotlib inline

UsageError: Line magic function `%` not found.


In [61]:
import sys
from PIL import ImageOps
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)


# np.set_printoptions(threshold=sys.maxsize)


# This cell contains the functions defined to read the images from the given directories,
# separate them into classes, and preprocess them so they can be used by thr model.

def image_cropper(img: np.array, height_percentage: int = 40, width_percentage: int = 40) -> np.array:
    """
    the image is cropped from both sides of the height and width
    :param img: the images as an np.array
    :param width_percentage: the percentage of the pixels from the width of the image we would like to crop
    :param height_percentage: the percentage of the pixels from the height of the image we would like to crop
    :return: cropped image as np.array
    """
    height, width = img.shape[0], img.shape[1]
    cropped_height_amount = height // 100 * height_percentage
    cropped_width_amount = height // 100 * width_percentage
    img = img[cropped_height_amount // 2:height - cropped_height_amount // 2,
          cropped_width_amount // 2:width - cropped_width_amount // 2]
    return img


def dynamic_image_cropper(img: np.array, region_of_interest=500) -> np.array:
    # print(img)
    height, width = img.shape[0], img.shape[1]
    center = [height // 2, width // 2]
    right_top_corner = [0, width - 1]
    left_top_corner = [0, 0]
    right_bottom_corner = [height - 1, width - 1]
    left_bottom_corner = [height - 1, 0]
    left_bottom_corner_mean = np.mean(img[left_bottom_corner[0] - region_of_interest:left_bottom_corner[0],
                                      left_bottom_corner[1]:left_bottom_corner[1] + region_of_interest])
    left_top_corner_mean = np.mean(img[left_top_corner[0]: region_of_interest + left_bottom_corner[0],
                                   left_bottom_corner[1]:left_bottom_corner[1] + region_of_interest])
    right_top_corner_mean = np.mean(img[right_top_corner[0]:right_top_corner[0] + region_of_interest,
                                    right_top_corner[1] - region_of_interest: right_top_corner[1]])
    right_bottom_corner_mean = np.mean(img[right_bottom_corner[0] - region_of_interest:right_bottom_corner[0],
                                       right_bottom_corner[1] - region_of_interest:right_bottom_corner[1]])
    center_mean = np.mean(img[center[0] - region_of_interest // 2:center[0] + region_of_interest // 2,
                          center[1] - region_of_interest // 2:center[1] + region_of_interest // 2])
    # print(np.array(
    #     [center_mean, left_top_corner_mean, left_bottom_corner_mean, right_top_corner_mean, right_bottom_corner_mean]))

    min_mean = np.argmin(np.array(
        [center_mean, left_top_corner_mean, left_bottom_corner_mean, right_top_corner_mean, right_bottom_corner_mean]))
    # print(min_mean)
    if min_mean == 0:
        return img[center[0] - region_of_interest // 2:center[0] + region_of_interest // 2,
               center[1] - region_of_interest // 2:center[1] + region_of_interest // 2]
    elif min_mean == 1:
        return img[left_top_corner[0]: region_of_interest + left_bottom_corner[0],
               left_bottom_corner[1]:left_bottom_corner[1] + region_of_interest]
    elif min_mean == 2:
        return img[left_bottom_corner[0] - region_of_interest:left_bottom_corner[0],
               left_bottom_corner[1]:left_bottom_corner[1] + region_of_interest]
    elif min_mean == 3:
        return img[right_top_corner[0]:right_top_corner[0] + region_of_interest,
               right_top_corner[1] - region_of_interest: right_top_corner[1]]
    elif min_mean == 4:
        return img[right_bottom_corner[0] - region_of_interest:right_bottom_corner[0],
               right_bottom_corner[1] - region_of_interest:right_bottom_corner[1]]

def dynamic_image_cropper2(img):
    # print(img)
    # print(img.shape)
    img = np.uint8(img)
    gray_negative = abs(255-img)
    img = gray_negative
    blur = cv2.GaussianBlur(img, (5,5), cv2.BORDER_DEFAULT)
    # img = img + (img-blur)
    edges= cv2.Canny(img, 100,200)
    contours, hierarchy= cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    sorted_contours= sorted(contours, key=cv2.contourArea, reverse= True)

    objects = []
    for (i,c) in enumerate(sorted_contours):
        x,y,w,h= cv2.boundingRect(c)
        new = img[y- 10:y+h + 10, x- 20:x+w + 10]
        new[new<125] = 0
        objects.append(new)
    biggest_object = np.array([[1,1],[1,1]])
    for obj in objects:
        if obj.shape[0] * obj.shape[1] > biggest_object.shape[0] * biggest_object.shape[1]:
            biggest_object = obj
    return biggest_object
    # cv2.imshow("test", biggest_object)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()



def image_scaler(img, new_height=20, new_width=20):
    """
    :param img: the images as an np.array
    :param new_height: the new height of the scaled image
    :param new_width: the new width of the scaled image
    :return: scaled image as np.array
    """
    img = Image.fromarray(img)
    img = img.resize((new_width, new_height))
    img = np.array(img)
    return img


def image_standardization(img, max_value=255):
    """
    changes the values of the image pixels to be between 0 and max_value
    :param max_value: max_value of the new pixels
    :param img: the images as an np.array
    :return: standardized image
    """
    # print(np.min(img), np.max(img))
    img = (img - np.min(img)) / (np.max(img) - np.min(img)) * max_value
    # img = (img // 16) * 16
    return img


def separate_classes(img_files, name_separation_chars=3, test_flag=False, classes_dict=None):
    """
    separates the images into classes based on their file names
    :param img_files: list of image file names
    :param name_separation_chars: number of characters in the name that should get used for separation
    :return: a list of classes for image names, a dict containing the name and the corresponding class number of images
    """
    names = list(set(img_name[:name_separation_chars] for img_name in img_files))
    if not test_flag:
        classes_dict = dict(enumerate(names))
        classes_dict = {v: k for k, v in classes_dict.items()}
    classes = []
    for img_name in img_files:
        classes.append(classes_dict[img_name[:name_separation_chars]])
    return classes, classes_dict


def get_files_list(data_dir, name_separation_chars=3, test_flag=False, classes_dict=None):
    """
    gets the list of image names from the directory
    :param data_dir: path to the image files
    :return: list of image names, their classes and the dict for classes
    """
    data = os.listdir(data_dir)
    ### print(data, len(data))
    classes, classes_dict = separate_classes(data, name_separation_chars=name_separation_chars, test_flag=test_flag,
                                             classes_dict=classes_dict)
    return data, classes, classes_dict


def image_reader(img_dir):
    """
    reads image in grayscale
    :return: img as np.array
    """
    img = cv2.imread(img_dir)
    gray= cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    return gray


def calculate_image_edges(img, total_extracted_features, threshold=100, peak_threshold=0.3, cutoff_percentage=20):
    img_edges = img.filter(ImageFilter.FIND_EDGES)
    # display(img_edges)
    edge_array = np.array(img_edges)
    edge_array = edge_array[4:edge_array.shape[0] - 4, 4:edge_array.shape[1] - 4]
    edge_array[edge_array < threshold] = 0
    # print(edge_array)
    edges_v = np.sum(edge_array, axis=0).astype(float)
    # print(edges_v.shape)
    edges_h = np.sum(edge_array, axis=1).astype(float)
    # print(edges_h.shape)
    # edges_sum = np.hstack(edges_v,edges_h)
    # print(edges_sum.shape)
    # print("edges_h1",edges_h)
    # M = 4
    # edges_v_copy = edges_v.copy()
    # for i in range(M // 2, len(edges_v) - M // 2):
    #     edges_v[i] = edges_v[i] - np.mean(edges_v_copy[i - M // 2:i + M // 2])
    # edges_v[:M // 2] = 0
    # edges_v[-M // 2:] = 0
    # edges_h_copy = edges_h.copy()
    # # print("edges_h1", edges_h_copy)
    # for j in range(M // 2, len(edges_h) - M // 2):
    #     # print(np.mean(edges_h_copy[j - M // 2:j + M // 2]))
    #     # print(edges_h[j])
    #     edges_h[j] = edges_h[j] - np.mean(edges_h_copy[j - M // 2:j + M // 2])
    # edges_h[:M // 2] = 0
    # edges_h[-M // 2:] = 0
    # cutoff = edge_array.shape[0] * cutoff_percentage // 100
    # edges_h = edges_h[cutoff:-cutoff]
    # edges_v = edges_v[cutoff:-cutoff]
    # print("edges_h2", edges_v)
    # peak_v = np.max(np.abs(edges_v))
    # peak_h = np.max(np.abs(edges_h))
    # # low_v = np.min(edges_v)
    # edges_v[np.abs(edges_v) < peak_v * peak_threshold] = 0
    # edges_h[np.abs(edges_h) < peak_h * peak_threshold] =

    edges_v_indexes = np.sort(edges_h.argsort()[-total_extracted_features // 2:][::-1])

    edges_v_extracted = edges_v[edges_v_indexes]
    # print(edges_v_extracted)
    edges_h_indexes = np.sort(edges_h.argsort()[-total_extracted_features // 2:][::-1])
    edges_h_extracted = edges_h[edges_h_indexes]

    # edges_v_extracted = np.array(edges_v_extracted)
    # edges_h_extracted = np.array(edges_h_extracted)
    # # # print(noise_v,noise_h)
    # sd_v = np.sqrt(np.var(noise_v))
    # sd_h = np.sqrt(np.var(noise_h))
    # # # print(sd_h,sd_v)
    # # # sd_h, sd_v = peak_h,peak_v
    # x = np.array([edges_v / sd_v, edges_h / sd_h]).reshape(1, -1)
    # scaler = MinMaxScaler()
    # x = np.array([scaler.fit_transform(edges_v_extracted.reshape(-1,1)), scaler.fit_transform(edges_h_extracted.reshape(-1,1))]).reshape(1, -1)
    # print(len(edges_v_extracted))
    # print(edges_v_extracted)
    x = np.array([edges_v_extracted, edges_h_extracted]).reshape(-1, 1)
    # print(x.shape)
    return x

def calculate_image_edges2(img, total_extracted_features, threshold=100, peak_threshold=0.3, cutoff_percentage=20):
    img = np.array(img)
    # blur
    # img = img + (img - cv2.GaussianBlur(img, (5,5), cv2.BORDER_DEFAULT))
    blur = cv2.GaussianBlur(img, (5,5), cv2.BORDER_DEFAULT)
    edge_array = cv2.Canny(image=blur, threshold1=100, threshold2=200, apertureSize= 3)
    # cv2.imshow("test", edge_array)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    edges_v = np.sum(edge_array, axis=0).astype(float)
    edges_h = np.sum(edge_array, axis=1).astype(float)
    # print(edges_h.shape)
    # print(edges_v.shape)
    edges_v_indexes = np.sort(edges_v.argsort()[-total_extracted_features // 2:][::-1])
    # print(edges_v_indexes)
    edges_v_extracted = edges_v[edges_v_indexes]

    edges_h_indexes = np.sort(edges_h.argsort()[-total_extracted_features // 2:][::-1])
    edges_h_extracted = edges_h[edges_h_indexes]
    # x = np.array([edges_v_extracted, edges_h_extracted]).reshape(-1, 1)
    x = np.array([edges_v_extracted, edges_h_extracted]).reshape(-1, 1)
    return x
    # print(sobelx)
    # cv2.imshow("test", sobelx)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    # print('--------')
    # print(sobely)
    # cv2.imshow("test", sobely)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    # print('--------')
    # print(sobelxy)
    # cv2.imshow("test", sobelxy)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    # print('--------')
    # print(edges)

def calculate_image_edges3(img, total_extracted_features, threshold=100, peak_threshold=0.3, cutoff_percentage=20):
    vertical = np.sum(img, axis = 0)
    horizontal = np.sum(img, axis = 1)
    # vertical[vertical<np.max(vertical) * 0.05] = 0
    # horizontal[horizontal<np.max(horizontal) * 0.05] = 0
    vertical[vertical == 0] = sorted(set(vertical))[1]
    horizontal[horizontal == 0] = sorted(set(horizontal))[1]
    edges_v_indexes = np.sort(vertical.argsort()[-total_extracted_features // 2:][::-1])
    # print(edges_v_indexes)
    edges_v_extracted = vertical[edges_v_indexes]
    edges_h_indexes = np.sort(horizontal.argsort()[-total_extracted_features // 2:][::-1])
    # print(edges_v_indexes)
    edges_h_extracted = horizontal[edges_h_indexes]
    x = np.array([edges_v_extracted, edges_h_extracted])
    # print(x)
    # print((x+1).shape)
    # x = np.log2(x + 1)
    return x



def get_data(path_to_training_data,
             class_name_chars=3,
             test_flag=False,
             classes_dict=None,
             augment_data=False,
             augments=[90, 180]
             ):
    data = []
    image_names, classes, classes_dict = get_files_list(path_to_training_data, class_name_chars, test_flag=test_flag,
                                                        classes_dict=classes_dict)
    new_classes = []
    for i, name in enumerate(image_names):
        img = image_reader(str(Path(path_to_training_data) / name))
        data.append(np.array(img))
        new_classes.append(classes[i])
        if augment_data:
            for value in augments:
                rotated = ndimage.rotate(img, value)
                rotated = np.array(rotated)
                rotated[rotated == 0] = np.mean(np.array(img)[0:20, 0:20])
                data.append(rotated)
                new_classes.append(classes[i])

    # converting classes list to a column array
    new_classes = np.array(new_classes).reshape(-1, 1)
    return np.array(data), new_classes, classes_dict


def preprocess_data(data,
                    standardization_flag=True,
                    standardization_max_value=255,
                    crop_flag=True,
                    crop_height_percentage=50,
                    crop_width_percentage=50,
                    scaling_flag=True,
                    scaling_height=100,
                    scaling_width=100,
                    total_extracted_features=64,
                    region_of_interest=500,
                    scaling = True,
                    scaler=None):
    new_data = []
    for img in data:
        if standardization_flag:
            img = image_standardization(img, standardization_max_value)
        if scaling_flag:
            img = image_scaler(img, scaling_height, scaling_width)
        if crop_flag:
            # print(img)
            # img = np.array(img)
            # img = dynamic_image_cropper(img, region_of_interest=region_of_interest)
            img = dynamic_image_cropper2(img)
            # display(Image.fromarray(img).convert('L'))


        # img = Image.fromarray(img).convert('L')
        img = calculate_image_edges3(img, total_extracted_features)
        img = img.flatten()
        # print(img)
        new_data.append(img)
    new_data = np.array(new_data)
    if scaling:
        if scaler is None:
            scaler = MinMaxScaler()
            scaler.fit(new_data)
        new_data = scaler.transform(new_data)
    return new_data, scaler
    # return new_data


region_of_interest = 600
scaling_height = 200
scaling_width = 200


def train_function(path_to_data,
                   class_name_chars=3,
                   test_flag=False,
                   classes_dict=None,
                   standardization_flag=True,
                   standardization_max_value=255,
                   crop_flag=True,
                   crop_height_percentage=15,
                   crop_width_percentage=15,
                   scaling_flag=True,
                   scaling_height=scaling_height,
                   scaling_width=scaling_width,
                   region_of_interest=region_of_interest,
                   total_extracted_features=64,
                   results=None,
                   scaling = True,
                   scaler=None):
    data, y_train, classes_dict = get_data(path_to_data, class_name_chars, test_flag, classes_dict)
    X_train, scaler = preprocess_data(data, standardization_flag=standardization_flag,
                                      standardization_max_value=standardization_max_value, crop_flag=crop_flag,
                                      crop_height_percentage=crop_height_percentage,
                                      crop_width_percentage=crop_width_percentage, scaling_flag=scaling_flag,
                                      scaling_height=scaling_height, scaling_width=scaling_width,
                                      total_extracted_features=total_extracted_features,
                                      region_of_interest=region_of_interest, scaling = scaling, scaler=scaler)
    # print(X_train)
    # train LogisticRegression model
    model = LogisticRegression(max_iter=5000000)
    # print(X_train)
    # print(X_train.shape)
    model.fit(X_train, y_train.ravel())

    print(f'\nX_train.shape = {X_train.shape}, y_train.shape = {len(y_train)}')
    y_pred = model.predict(X_train)

    # Reporting the stats from the model
    print(f'\nConfusion Matrix = \n{confusion_matrix(y_train, y_pred)}')
    print(f'\nAccuracy Score = {accuracy_score(y_train, y_pred)}')
    print(f'Model Coefficient Shape = {model.coef_.shape}')
    if results is not None:
        results.append(accuracy_score(y_train, y_pred))
    return model, classes_dict, results, scaler


model, classes_dict, results, scaler = train_function('../../data/Lego_dataset_2/training/')


C:\Users\AshesFall\AppData\Local\Temp\ipykernel_13076\4017461687.py:311: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(data), new_classes, classes_dict


[[ 7968  8155  8393  8764  9105  9430  9762 10035 10552 10585 10764 11059
  11147 11308 11354 11389 11378 11283 11323 11286 11171 10840 10768 10609
  10428 10299 10078  9719  9385  8891  8596  8221]
 [ 8032  8250  8293  8526  8726  8913  9123  9374  9789 10108 10128 10286
  10695 10786 10977 11041 11071 11122 10992 10787 10741 10632 10417 10342
  10234  9970  9712  9594  9169  8864  8597  8261]]
[[ 7767  7931  8175  8531  8959  9320  9677  9767 10203 10510 10534 10827
  10932 11114 11144 11174 11301 11216 11129 11093 10842 10643 10578 10414
  10332 10077  9879  9601  9139  8771  8392  7845]
 [ 7821  8030  8215  8438  8516  8707  9021  9181  9459  9879  9886 10104
  10360 10563 10614 10805 10833 10728 10604 10661 10471 10384 10301 10074
   9963  9683  9454  9194  8884  8575  8233  7871]]
[[6440 6576 6715 6752 6891 6900 6908 6951 6966 7353 7569 7737 7793 8186
  8200 8159 8126 8115 8020 7783 7872 7724 7371 7272 7239 7331 7266 7101
  7097 7073 7017 6702]
 [5980 6080 6268 6282 6571 6594 669

In [76]:
# The test function uses the get_data function as a base to read all the images from the given directory, get their classes based on the classes_dict from the training function, prepare the dataset, and then predict the classes of the images.

def test_function(path_to_data,
                  model,
                  class_name_chars=3,
                  test_flag=False,
                  classes_dict=None,
                  standardization_flag=True,
                  standardization_max_value=255,
                  crop_flag=True,
                  crop_height_percentage=10,
                  crop_width_percentage=10,
                  scaling_flag=True,
                  scaling_height=scaling_height,
                  scaling_width=scaling_width,
                  region_of_interest=region_of_interest,
                  total_extracted_features=64,
                  results=None,
                  scaling = True,
                  scaler = None):
    data, y_test, classes_dict = get_data(path_to_data, class_name_chars, test_flag, classes_dict)
    X_test, scaler = preprocess_data(data, standardization_flag=standardization_flag,
                             standardization_max_value=standardization_max_value, crop_flag=crop_flag,
                             crop_height_percentage=crop_height_percentage,
                             crop_width_percentage=crop_width_percentage, scaling_flag=scaling_flag,
                             scaling_height=scaling_height, scaling_width=scaling_width,
                             total_extracted_features=total_extracted_features, region_of_interest=region_of_interest, scaling = scaling, scaler = scaler)
    # train LogisticRegression model
    print(f'\nX_test.shape = {X_test.shape}, y_test.shape = {len(y_test)}')
    y_pred = model.predict(X_test)
    print(classes_dict)
    # print(y_pred.tolist())
    print(np.where(y_pred != y_test.flatten())[0] + 1)
    # Reporting the stats from the model
    print(f'\nConfusion Matrix = \n{confusion_matrix(y_test, y_pred)}')
    print(f'\nAccuracy Score = {accuracy_score(y_test, y_pred)}')
    print(f'Model Coefficient Shape = {model.coef_.shape}')
    if results is not None:
        results.append(accuracy_score(y_test, y_pred))
    return results


results = test_function('../../data/Lego_dataset_2/testing/', model, test_flag=True, classes_dict=classes_dict, scaler=scaler)

C:\Users\AshesFall\AppData\Local\Temp\ipykernel_13076\4017461687.py:311: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(data), new_classes, classes_dict


[[ 7152  7516  7845  8090  8528  8895  9116  9205  9461  9670  9639  9835
   9318  8801  8565  9107  9374  9324  9838 10177 10069  9942  9842  9835
   9673  9428  9120  8685  8183  7769  7526  7128]
 [ 7109  7365  7738  7953  8360  8504  8677  8875  9024  9361  9473  9437
   9514  9514  9487  8934  8399  8248  8282  8493  8937  9215  9070  8955
   8743  8605  8489  8164  7936  7558  7299  6909]]
[[ 7286  7360  7671  7946  8259  8466  8662  8923  9379  9585  9640  9816
  10157 10258 10328 10525 10577 10369 10177 10093  9961  9742  9653  9447
   9361  9012  8926  8700  8395  7990  7809  7376]
 [ 7125  7315  7534  7658  7987  8204  8519  9132  9313  9433  9644  9932
   9968 10168 10269 10367  9995  9882  9839  9612  9446  9425  9210  9034
   8944  8760  8522  8400  7963  7809  7604  7199]]
[[ 9605 10077 10431 10719 10820 10892 10973 11232 11362 11439 11707 11847
  11859 11902 12133 12181 12184 12175 12180 12165 12094 11943 11919 11799
  11469 11230 11014 10908 10672 10516 10060  9722]
 [ 

In [64]:
results